In [1]:
import ITensors:op
import Pkg; Pkg.add("HDF5")
import Pkg; Pkg.add("Plots")
import Pkg; Pkg.add("PyFormattedStrings")
import Pkg; Pkg.add("LinearAlgebra")
import Pkg; Pkg.add("LaTeXStrings")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [2]:
import Pkg; Pkg.add("ITensorTDVP")
using ITensors, ITensorMPS
import ITensors:op
using HDF5
using Plots
using PyFormattedStrings
using LinearAlgebra
using ITensorTDVP: tdvp

ITensors.disable_warn_order()

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


14

In [3]:
#include("overload_Spin=3h.jl")
# overloadings for SiteType 3/2
#----------------------------------------------
ITensors.space(::SiteType"S=3/2") = 4

# operator Sz
#-------------------
ITensors.op(::OpName"Sz",::SiteType"S=3/2") =
[+3/2   0    0    0
  0    +1/2  0    0
  0     0   -1/2  0
  0     0    0   -3/2]

# operator S+
#-------------------
ITensors.op(::OpName"S+",::SiteType"S=3/2") =
[0  √3  0   0 
 0  0   √4  0
 0  0   0   √3
 0  0   0   0]

# operator S-
#-------------------
ITensors.op(::OpName"S-",::SiteType"S=3/2") =
[0  0   0   0
 √3 0   0   0
 0  √4  0   0
 0  0   √3  0]

# z projection states
#-----------------------------------------
ITensors.state(::StateName"+3/2",::SiteType"S=3/2") = [1, 0, 0, 0]
ITensors.state(::StateName"+1/2",::SiteType"S=3/2") = [0, 1, 0, 0]
ITensors.state(::StateName"-1/2",::SiteType"S=3/2") = [0, 0, 1, 0]
ITensors.state(::StateName"-3/2",::SiteType"S=3/2") = [0, 0, 0, 1]

In [4]:
# construction of the MPO hamiltonian
#------------------------------------------
function Hamiltonian(sites)
    ops = OpSum()

    N = length(sites)
    # spin-spin interaction
    for k=1:N-1
        ops += J,   "Sz",k, "Sz",k+1
        ops += J/2, "S+",k, "S-",k+1
        ops += J/2, "S-",k, "S+",k+1
    end
    #make it a ring
    ops += J,   "Sz",N, "Sz",1
    ops += J/2, "S+",N, "S-",1
    ops += J/2, "S-",N, "S+",1

    # spin-B interaction
    for k=1:N
        ops += -B, "Sz",k
    end
    
    H = MPO(ops,sites)
    #--------------------------
    return H
end

Hamiltonian (generic function with 1 method)

In [5]:
function Hamiltonian1(sites)
    ops = OpSum()

    N = length(sites)
    # spin-spin interaction
    for k=1:N-1
        ops += J,   "Sz",k, "Sz",k+1
        ops += J/2, "S+",k, "S-",k+1
        ops += J/2, "S-",k, "S+",k+1
    end
    #make it a ring
    ops += J,   "Sz",N, "Sz",1
    ops += J/2, "S+",N, "S-",1
    ops += J/2, "S-",N, "S+",1

    # spin-B interaction
    for k=1:2
        ops += -B, "Sz",k
    end
    ops += -B1, "Sz",3
    
    for k=4:N
        ops += -B, "Sz",k
    end
    
    H = MPO(ops,sites)
    #--------------------------
    return H
end

Hamiltonian1 (generic function with 1 method)

In [6]:

#four point RDM of sites AB, CD
# AB adjacent, CD adjacent
#------------------------------------------
function get_4pdm(wf::MPS,locs)
    psi = copy(wf)
    A,C = locs

    orthogonalize!(psi,A)
    ket = psi[A]
    for k in A+1:C+1
        ket *= psi[k]
    end
    rho = prime(ket,"Site") * dag(ket)

    if C-A>2
        inds_list = inds(rho)
        trinds1 = collect(3 : C-A)
        trinds2 = trinds1[end] .+ trinds1 .+ 2
        for idx in 1:length(trinds1)
            rho = rho*delta(inds_list[trinds1[idx]],inds_list[trinds2[idx]])
        end
    end
    #-------------------
    return rho
end

get_4pdm (generic function with 1 method)

In [7]:
#   calc_Eigenvalues of density matrix
#-----------------------------------
function calc_Eigenvalues(density_matrix)
    egn_val,_ = eigen(density_matrix,ishermitian=true)
    #-----------------------------------
    return diag(array(egn_val))
end


#   eigenvalue based generic properties of a density matrix
#-----------------------------------
function calc_Norm(density_matrix)
    rho = copy(density_matrix)

    egn_val = calc_Eigenvalues(rho)
    Norm = sum(egn_val)
    Purity = sum(egn_val.^2)

    #-----------------------------------
    return [Norm,Purity]
end

#   eigenvalue based generic properties of a density matrix
#-----------------------------------
function calc_SvN(density_matrix)
    rho = copy(density_matrix)

    egn_val = calc_Eigenvalues(rho)
    SvN = sum( [ - lam*log(lam) for lam in egn_val if lam > 0 ] )

    #-----------------------------------
    return SvN
end


calc_SvN (generic function with 1 method)

In [8]:
#define the parameters of the hamiltonian and constuct MPO
#----------------------------------------------------------------------------
J = 1 
B = 0.1
B1 = 0.001
nsweeps = 20
maxdim = [100, 200, 500, 1000, 2000, 5000, 10000, 25580]
cutoff = [1E-11]

#--------------------
N = 14
#--------------------

site_list = collect(1:N) 
sites = siteinds( k->isodd(k) ? "S=1/2" : "S=3/2", N ) 
#sites = siteinds("S=1/2", N)

H = Hamiltonian(sites) 

# ground state and first excited state
#--------------------------------------------
energy0,psi0 = dmrg(H,random_mps(sites);nsweeps,maxdim,cutoff,ishermitian=true) ;
#energy1,psi1 = dmrg(H,[psi0],random_mps(sites);nsweeps,maxdim,cutoff,ishermitian=true) ;

After sweep 1 energy=-12.956355785972532  maxlinkdim=8 maxerr=1.44E-16 time=32.232
After sweep 2 energy=-14.14742039151256  maxlinkdim=32 maxerr=2.81E-16 time=0.253
After sweep 3 energy=-14.42954305649987  maxlinkdim=128 maxerr=9.31E-12 time=1.154
After sweep 4 energy=-14.463578095797548  maxlinkdim=329 maxerr=9.96E-12 time=5.561
After sweep 5 energy=-14.469217326547641  maxlinkdim=420 maxerr=9.99E-12 time=13.513
After sweep 6 energy=-14.470335038306326  maxlinkdim=325 maxerr=9.98E-12 time=10.749
After sweep 7 energy=-14.47058293137152  maxlinkdim=241 maxerr=9.93E-12 time=6.536
After sweep 8 energy=-14.47064073941498  maxlinkdim=184 maxerr=9.93E-12 time=7.413
After sweep 9 energy=-14.470654258941659  maxlinkdim=144 maxerr=9.88E-12 time=3.142
After sweep 10 energy=-14.470657440871832  maxlinkdim=109 maxerr=9.87E-12 time=2.052
After sweep 11 energy=-14.470658198418398  maxlinkdim=87 maxerr=9.87E-12 time=0.890
After sweep 12 energy=-14.470658382452687  maxlinkdim=76 maxerr=9.96E-12 time=0

In [9]:
H1 = Hamiltonian1(sites) 
ϕ = tdvp(
    H1,
    2.0im,
    psi0;
    time_step=1.0im,
    maxdim=30,
    cutoff=1e-10,
    normalize=true,
    reverse_step=false,
    outputlevel=1,
  )

After sweep 1: maxlinkdim=30 maxerr=6.10E-08 current_time=0.0 + 1.0im time=45.055
After sweep 2: maxlinkdim=30 maxerr=1.88E-07 current_time=0.0 + 2.0im time=4.058


MPS
[1] ((dim=2|id=657|"Link,l=1"), (dim=2|id=868|"S=1/2,Site,n=1"))
[2] ((dim=8|id=502|"Link,l=2"), (dim=4|id=200|"S=3/2,Site,n=2"), (dim=2|id=657|"Link,l=1"))
[3] ((dim=15|id=681|"Link,l=3"), (dim=2|id=944|"S=1/2,Site,n=3"), (dim=8|id=502|"Link,l=2"))
[4] ((dim=30|id=689|"Link,l=4"), (dim=4|id=858|"S=3/2,Site,n=4"), (dim=15|id=681|"Link,l=3"))
[5] ((dim=30|id=14|"Link,l=5"), (dim=2|id=63|"S=1/2,Site,n=5"), (dim=30|id=689|"Link,l=4"))
[6] ((dim=30|id=679|"Link,l=6"), (dim=4|id=401|"S=3/2,Site,n=6"), (dim=30|id=14|"Link,l=5"))
[7] ((dim=30|id=631|"Link,l=7"), (dim=2|id=151|"S=1/2,Site,n=7"), (dim=30|id=679|"Link,l=6"))
[8] ((dim=30|id=557|"Link,l=8"), (dim=4|id=635|"S=3/2,Site,n=8"), (dim=30|id=631|"Link,l=7"))
[9] ((dim=30|id=319|"Link,l=9"), (dim=2|id=770|"S=1/2,Site,n=9"), (dim=30|id=557|"Link,l=8"))
[10] ((dim=30|id=688|"Link,l=10"), (dim=4|id=994|"S=3/2,Site,n=10"), (dim=30|id=319|"Link,l=9"))
[11] ((dim=23|id=530|"Link,l=11"), (dim=2|id=495|"S=1/2,Site,n=11"), (dim=30|id=688|"Lin

In [10]:
exp_Sz = expect( ϕ, "Sz" )

14-element Vector{Float64}:
 -0.3582545757228024
  1.361121354185929
 -0.36390849557649024
  1.3620265184199694
 -0.3587444385692811
  1.3577550013240485
 -0.3577578194653766
  1.3576224294841572
 -0.35742206971642004
  1.3568851517475085
 -0.35717754804924207
  1.3571797092875473
 -0.35720773417850504
  1.3578824580635072

In [11]:
sum=0
for i in 1:length(exp_Sz)
    sum+=exp_Sz[i]
end

sum

6.99999994123455

In [12]:
using Plots
using LaTeXStrings
plot( expect(ϕ,"Sz"),marker=:circle )

GKS: dlopen(/Users/Hank/.julia/artifacts/66ebf8a541da7166677a6e2ac6b8277592d32044/lib/cairoplugin.so, 1): Library not loaded: @rpath/libcairo.2.dylib
  Referenced from: /Users/Hank/.julia/artifacts/66ebf8a541da7166677a6e2ac6b8277592d32044/lib/cairoplugin.so
  Reason: Incompatible library version: cairoplugin.so requires version 11603.0.0 or later, but libcairo.2.dylib provides version 2.0.0


In [13]:
# TDVP parameters
time_step = 1.0
maxdim = 30
cutoff = 1e-10
normalize = true
reverse_step = false
outputlevel = 1


# Initialize lists to store total times and sums of expectation values
total_times = [1.0 + (i-1)*time_step for i in 1:15]
sum_values = []

for total_time in total_times
    # Evolve the state
    psi = tdvp(H1, total_time, psi0; time_step=time_step, maxdim=maxdim, cutoff=cutoff, normalize=normalize, reverse_step=reverse_step, outputlevel=outputlevel)
    
    # Compute the expectation value of Sz
    exp_Sz = expect(psi, "Sz")
    
    # Sum the expectation values
    sum=0
    
    for i in 1:length(exp_Sz)
        sum+=exp_Sz[i]
    end
    
    # Store the sum
    push!(sum_values, sum)
end


# Plot the sum of expectation values vs total time
plot(abs.(total_times), sum_values, xlabel="time", ylabel="< Sz>", lw=2, label="Sum(Sz)")



After sweep 1: maxlinkdim=30 maxerr=5.98E-06 current_time=1.0 time=13.214
After sweep 1: maxlinkdim=30 maxerr=5.98E-06 current_time=1.0 time=2.205
After sweep 2: maxlinkdim=30 maxerr=1.33E-07 current_time=2.0 time=1.87
After sweep 1: maxlinkdim=30 maxerr=5.98E-06 current_time=1.0 time=2.423
After sweep 2: maxlinkdim=30 maxerr=1.33E-07 current_time=2.0 time=1.875
After sweep 3: maxlinkdim=30 maxerr=8.83E-08 current_time=3.0 time=1.896
After sweep 1: maxlinkdim=30 maxerr=5.98E-06 current_time=1.0 time=2.287
After sweep 2: maxlinkdim=30 maxerr=1.33E-07 current_time=2.0 time=2.053
After sweep 3: maxlinkdim=30 maxerr=8.83E-08 current_time=3.0 time=1.892
After sweep 4: maxlinkdim=30 maxerr=4.94E-08 current_time=4.0 time=1.895
After sweep 1: maxlinkdim=30 maxerr=5.98E-06 current_time=1.0 time=2.298
After sweep 2: maxlinkdim=30 maxerr=1.33E-07 current_time=2.0 time=1.883
After sweep 3: maxlinkdim=30 maxerr=8.83E-08 current_time=3.0 time=1.892
After sweep 4: maxlinkdim=30 maxerr=4.94E-08 curren

After sweep 9: maxlinkdim=5 maxerr=7.79E-11 current_time=9.0 time=0.357
After sweep 10: maxlinkdim=3 maxerr=2.47E-11 current_time=10.0 time=0.302
After sweep 11: maxlinkdim=2 maxerr=8.14E-11 current_time=11.0 time=0.351
After sweep 12: maxlinkdim=2 maxerr=1.02E-12 current_time=12.0 time=0.532
After sweep 13: maxlinkdim=2 maxerr=2.86E-13 current_time=13.0 time=0.278
After sweep 14: maxlinkdim=2 maxerr=5.21E-11 current_time=14.0 time=0.303
After sweep 15: maxlinkdim=1 maxerr=1.35E-13 current_time=15.0 time=0.112


In [14]:
sum_values

15-element Vector{Any}:
   6.999797241877385
   6.974751960071389
   4.828155449937961
  -2.3362738606647744
  -9.972414951423088
 -12.724619391310016
 -13.821798015240297
 -13.985387121451666
 -13.998806630191625
 -13.999897410975198
 -13.999986851723985
 -13.999996734807574
 -13.999999040905765
 -13.999999723355568
 -13.99999984208936

In [15]:
using ITensors, ITensorMPS
ITensors.disable_warn_order()
let
A = 1
D = 4
N = 20

  # Make an array of N Index objects with alternating
  # "S=1/2" and "S=1" tags on odd versus even sites
  # (The first argument n->isodd(n) ... is an
  # on-the-fly function mapping integers to strings)
  sites = siteinds(n->isodd(n) ? "S=1/2" : "S=3/2",N)

  # Couplings between spin-half and
  # spin-one sites:
  J=1
  B=0.1

  ops = OpSum()
  for k=1:N-1
        ops += J,   "Sz",k, "Sz",k+1
        ops += J/2, "S+",k, "S-",k+1
        ops += J/2, "S-",k, "S+",k+1
    end
    
    #make it a ring
    ops += J,   "Sz",N, "Sz",1
    ops += J/2, "S+",N, "S-",1
    ops += J/2, "S-",N, "S+",1

    # spin-B interaction
    for k=1:N
        ops += -B, "Sz",k
    end
    
    H = MPO(ops,sites)
    
nsweeps = 20
maxdim = [5,50,100,150,200,250,300,400,500]
cutoff = [1E-11]
    
# Create the density matrix rho
a1 = 0
N1 = 0
a2 = 0
N2 = 0
a3 = 0
N3 = 0
a4 = 0
N4 = 0
a5 = 0
N5 = 0
a6 = 0
N6 = 0
a7 = 0
N7 = 0
orthogonalize!(ϕ,A)
ket1 = ϕ[A]
for i=A+1:D
    ket1 *= ϕ[i]
end
bra1 = dag(ket1)
rho = prime(ket1,"Site")*bra1
#@show inds(rho)  #<-- this shows that the indices of rho corresponding to the site between B and C are the 3rd and 7th element of the inds(rho) vector.    
rho_ABC3 = rho      # delta(inds,inds) traces out the given inds (see the documentation)  
#rho_ABC3 = rho_ABC*delta(inds(rho)[4],inds(rho)[10])   
#rho_ABC3 = rho_ABC1*delta(inds(rho)[5],inds(rho)[12])  
#rho_ABC3 = rho_ABC2*delta(inds(rho)[6],inds(rho)[14]) 
#da, ua = eigen(rho_ABC1)


oldinds = inds(rho_ABC3)
newinds = [oldinds[5],oldinds[6],oldinds[3],oldinds[4],oldinds[1], oldinds[2], oldinds[7], oldinds[8]]
newinds1 = [oldinds[5],oldinds[2],oldinds[7],oldinds[4],oldinds[1], oldinds[6], oldinds[3], oldinds[8]]
newinds2 = [oldinds[5],oldinds[2],oldinds[3],oldinds[8],oldinds[1], oldinds[6], oldinds[7], oldinds[4]]
newinds3 = [oldinds[5],oldinds[2],oldinds[3],oldinds[4],oldinds[1], oldinds[6], oldinds[7], oldinds[8]]
newinds4 = [oldinds[1],oldinds[6],oldinds[3],oldinds[4],oldinds[5], oldinds[2], oldinds[7], oldinds[8]]
newinds5 = [oldinds[1],oldinds[2],oldinds[7],oldinds[4],oldinds[5], oldinds[6], oldinds[3], oldinds[8]]
newinds6 = [oldinds[1],oldinds[2],oldinds[3],oldinds[8],oldinds[5], oldinds[6], oldinds[7], oldinds[4]]


rho_pt = swapinds(rho_ABC3,oldinds,newinds)
rho_pt1 = swapinds(rho_ABC3,oldinds,newinds1)
rho_pt2 = swapinds(rho_ABC3,oldinds,newinds2)
rho_pt3 = swapinds(rho_ABC3,oldinds,newinds3)
rho_pt4 = swapinds(rho_ABC3,oldinds,newinds4)
rho_pt5 = swapinds(rho_ABC3,oldinds,newinds5)
rho_pt6 = swapinds(rho_ABC3,oldinds,newinds6)

# Compute the eigenvalues of the partially transposed density matrix
da, ua = eigen(rho_pt)
da1, ua1 = eigen(rho_pt1)
da2, ua2 = eigen(rho_pt2)
da3, ua3 = eigen(rho_pt3)
da4, u4 = eigen(rho_pt4)
da5, ua5 = eigen(rho_pt5)
da6, ua6 = eigen(rho_pt6)

# Calculate the negativity
for j = 1:length(diag(real(da)))
    a1 += diag(real(da))[j]
end


for j = 1:length(diag(real(da)))
    N1 += 0.5 * (abs(diag(real(da))[j]) - diag(real(da))[j])
end
L1 = log(1 + 2 * N1)


for j = 1:length(diag(real(da1)))
    N2 += 0.5 * (abs(diag(real(da1))[j]) - diag(real(da1))[j])
end
L2 = log(1 + 2 * N2)

for j = 1:length(diag(real(da2)))
    N3 += 0.5 * (abs(diag(real(da2))[j]) - diag(real(da2))[j])
end
L3 = log(1 + 2 * N3)


for j = 1:length(diag(real(da3)))
    N4 += 0.5 * (abs(diag(real(da3))[j]) - diag(real(da3))[j])
end
L4 = log(1 + 2 * N4)


for j = 1:length(diag(real(da4)))
    N5 += 0.5 * (abs(diag(real(da4))[j]) - diag(real(da4))[j])
end
L5 = log(1 + 2 * N5)

for j = 1:length(diag(real(da5)))
    N6 += 0.5 * (abs(diag(real(da5))[j]) - diag(real(da5))[j])
end
L6 = log(1 + 2 * N6)


for j = 1:length(diag(real(da6)))
    N7 += 0.5 * (abs(diag(real(da6))[j]) - diag(real(da6))[j])
end
L7 = log(1 + 2 * N7)

L00=(L1*L2*L3*L4*L5*L6*L7)^(1/7)
@show L00;
  
end

L00 = 0.4978856855988925


0.4978856855988925

In [16]:
using ITensors, ITensorMPS
ITensors.disable_warn_order()
let
A = 1
D = 5
N = 20

  # Make an array of N Index objects with alternating
  # "S=1/2" and "S=1" tags on odd versus even sites
  # (The first argument n->isodd(n) ... is an
  # on-the-fly function mapping integers to strings)
  sites = siteinds(n->isodd(n) ? "S=1/2" : "S=3/2",N)

  # Couplings between spin-half and
  # spin-one sites:
  J=1
  B=0.1

  ops = OpSum()
  for k=1:N-1
        ops += J,   "Sz",k, "Sz",k+1
        ops += J/2, "S+",k, "S-",k+1
        ops += J/2, "S-",k, "S+",k+1
    end
    
    #make it a ring
    ops += J,   "Sz",N, "Sz",1
    ops += J/2, "S+",N, "S-",1
    ops += J/2, "S-",N, "S+",1

    # spin-B interaction
    for k=1:N
        ops += -B, "Sz",k
    end
    
    H = MPO(ops,sites)
    
nsweeps = 20
maxdim = [5,50,100,150,200,250,300,400,500]
cutoff = [1E-11]
    
# Create the density matrix rho
a1 = 0
N1 = 0
a2 = 0
N2 = 0
a3 = 0
N3 = 0
a4 = 0
N4 = 0
a5 = 0
N5 = 0
a6 = 0
N6 = 0
a7 = 0
N7 = 0
orthogonalize!(ϕ,A)
ket1 = ϕ[A]
for i=A+1:D
    ket1 *= ϕ[i]
end
bra1 = dag(ket1)
rho = prime(ket1,"Site")*bra1
#@show inds(rho)  #<-- this shows that the indices of rho corresponding to the site between B and C are the 3rd and 7th element of the inds(rho) vector.    
rho_ABC3 = rho * delta(inds(rho)[3],inds(rho)[8])     # delta(inds,inds) traces out the given inds (see the documentation)  
#rho_ABC3 = rho_ABC*delta(inds(rho)[4],inds(rho)[10])   
#rho_ABC3 = rho_ABC1*delta(inds(rho)[5],inds(rho)[12])  
#rho_ABC3 = rho_ABC2*delta(inds(rho)[6],inds(rho)[14]) 
#da, ua = eigen(rho_ABC1)


oldinds = inds(rho_ABC3)
newinds = [oldinds[5],oldinds[6],oldinds[3],oldinds[4],oldinds[1], oldinds[2], oldinds[7], oldinds[8]]
newinds1 = [oldinds[5],oldinds[2],oldinds[7],oldinds[4],oldinds[1], oldinds[6], oldinds[3], oldinds[8]]
newinds2 = [oldinds[5],oldinds[2],oldinds[3],oldinds[8],oldinds[1], oldinds[6], oldinds[7], oldinds[4]]
newinds3 = [oldinds[5],oldinds[2],oldinds[3],oldinds[4],oldinds[1], oldinds[6], oldinds[7], oldinds[8]]
newinds4 = [oldinds[1],oldinds[6],oldinds[3],oldinds[4],oldinds[5], oldinds[2], oldinds[7], oldinds[8]]
newinds5 = [oldinds[1],oldinds[2],oldinds[7],oldinds[4],oldinds[5], oldinds[6], oldinds[3], oldinds[8]]
newinds6 = [oldinds[1],oldinds[2],oldinds[3],oldinds[8],oldinds[5], oldinds[6], oldinds[7], oldinds[4]]


rho_pt = swapinds(rho_ABC3,oldinds,newinds)
rho_pt1 = swapinds(rho_ABC3,oldinds,newinds1)
rho_pt2 = swapinds(rho_ABC3,oldinds,newinds2)
rho_pt3 = swapinds(rho_ABC3,oldinds,newinds3)
rho_pt4 = swapinds(rho_ABC3,oldinds,newinds4)
rho_pt5 = swapinds(rho_ABC3,oldinds,newinds5)
rho_pt6 = swapinds(rho_ABC3,oldinds,newinds6)

# Compute the eigenvalues of the partially transposed density matrix
da, ua = eigen(rho_pt)
da1, ua1 = eigen(rho_pt1)
da2, ua2 = eigen(rho_pt2)
da3, ua3 = eigen(rho_pt3)
da4, u4 = eigen(rho_pt4)
da5, ua5 = eigen(rho_pt5)
da6, ua6 = eigen(rho_pt6)

# Calculate the negativity
for j = 1:length(diag(real(da)))
    a1 += diag(real(da))[j]
end


for j = 1:length(diag(real(da)))
    N1 += 0.5 * (abs(diag(real(da))[j]) - diag(real(da))[j])
end
L1 = log(1 + 2 * N1)


for j = 1:length(diag(real(da1)))
    N2 += 0.5 * (abs(diag(real(da1))[j]) - diag(real(da1))[j])
end
L2 = log(1 + 2 * N2)

for j = 1:length(diag(real(da2)))
    N3 += 0.5 * (abs(diag(real(da2))[j]) - diag(real(da2))[j])
end
L3 = log(1 + 2 * N3)


for j = 1:length(diag(real(da3)))
    N4 += 0.5 * (abs(diag(real(da3))[j]) - diag(real(da3))[j])
end
L4 = log(1 + 2 * N4)


for j = 1:length(diag(real(da4)))
    N5 += 0.5 * (abs(diag(real(da4))[j]) - diag(real(da4))[j])
end
L5 = log(1 + 2 * N5)

for j = 1:length(diag(real(da5)))
    N6 += 0.5 * (abs(diag(real(da5))[j]) - diag(real(da5))[j])
end
L6 = log(1 + 2 * N6)


for j = 1:length(diag(real(da6)))
    N7 += 0.5 * (abs(diag(real(da6))[j]) - diag(real(da6))[j])
end
L7 = log(1 + 2 * N7)

L11=(L1*L2*L3*L4*L5*L6*L7)^(1/7)
@show L11;
    
    
end   

L11 = 0.23286051319570816


0.23286051319570816

In [17]:
using ITensors, ITensorMPS
ITensors.disable_warn_order()
let
A = 1
D = 6
N = 20

  sites = siteinds(n->isodd(n) ? "S=1/2" : "S=3/2",N)

  # Couplings between spin-half and
  # spin-one sites:
  J=1
  B=0.1

  ops = OpSum()
  for k=1:N-1
        ops += J,   "Sz",k, "Sz",k+1
        ops += J/2, "S+",k, "S-",k+1
        ops += J/2, "S-",k, "S+",k+1
    end
    
    #make it a ring
    ops += J,   "Sz",N, "Sz",1
    ops += J/2, "S+",N, "S-",1
    ops += J/2, "S-",N, "S+",1

    # spin-B interaction
    for k=1:N
        ops += -B, "Sz",k
    end
    
    H = MPO(ops,sites)
    
    
nsweeps = 20
maxdim = [5,50,100,150,200,250,300,400,500]
cutoff = [1E-11]

# Create the density matrix rho
a1 = 0
N1 = 0
a2 = 0
N2 = 0
a3 = 0
N3 = 0
a4 = 0
N4 = 0
a5 = 0
N5 = 0
a6 = 0
N6 = 0
a7 = 0
N7 = 0
orthogonalize!(ϕ,A)
ket1 = ϕ[A]
for i=A+1:D
    ket1 *= ϕ[i]
end
bra1 = dag(ket1)
rho = prime(ket1,"Site")*bra1
#@show inds(rho) 
rho_ABC = rho * delta(inds(rho)[3],inds(rho)[9])    
rho_ABC3 = rho_ABC*delta(inds(rho)[4],inds(rho)[10])   



oldinds = inds(rho_ABC3)
newinds = [oldinds[5],oldinds[6],oldinds[3],oldinds[4],oldinds[1], oldinds[2], oldinds[7], oldinds[8]]
newinds1 = [oldinds[5],oldinds[2],oldinds[7],oldinds[4],oldinds[1], oldinds[6], oldinds[3], oldinds[8]]
newinds2 = [oldinds[5],oldinds[2],oldinds[3],oldinds[8],oldinds[1], oldinds[6], oldinds[7], oldinds[4]]
newinds3 = [oldinds[5],oldinds[2],oldinds[3],oldinds[4],oldinds[1], oldinds[6], oldinds[7], oldinds[8]]
newinds4 = [oldinds[1],oldinds[6],oldinds[3],oldinds[4],oldinds[5], oldinds[2], oldinds[7], oldinds[8]]
newinds5 = [oldinds[1],oldinds[2],oldinds[7],oldinds[4],oldinds[5], oldinds[6], oldinds[3], oldinds[8]]
newinds6 = [oldinds[1],oldinds[2],oldinds[3],oldinds[8],oldinds[5], oldinds[6], oldinds[7], oldinds[4]]


rho_pt = swapinds(rho_ABC3,oldinds,newinds)
rho_pt1 = swapinds(rho_ABC3,oldinds,newinds1)
rho_pt2 = swapinds(rho_ABC3,oldinds,newinds2)
rho_pt3 = swapinds(rho_ABC3,oldinds,newinds3)
rho_pt4 = swapinds(rho_ABC3,oldinds,newinds4)
rho_pt5 = swapinds(rho_ABC3,oldinds,newinds5)
rho_pt6 = swapinds(rho_ABC3,oldinds,newinds6)


da, ua = eigen(rho_pt)
da1, ua1 = eigen(rho_pt1)
da2, ua2 = eigen(rho_pt2)
da3, ua3 = eigen(rho_pt3)
da4, u4 = eigen(rho_pt4)
da5, ua5 = eigen(rho_pt5)
da6, ua6 = eigen(rho_pt6)

# Calculate the negativity
for j = 1:length(diag(real(da)))
    a1 += diag(real(da))[j]
end


for j = 1:length(diag(real(da)))
    N1 += 0.5 * (abs(diag(real(da))[j]) - diag(real(da))[j])
end
L1 = log(1 + 2 * N1)


for j = 1:length(diag(real(da1)))
    N2 += 0.5 * (abs(diag(real(da1))[j]) - diag(real(da1))[j])
end
L2 = log(1 + 2 * N2)

for j = 1:length(diag(real(da2)))
    N3 += 0.5 * (abs(diag(real(da2))[j]) - diag(real(da2))[j])
end
L3 = log(1 + 2 * N3)


for j = 1:length(diag(real(da3)))
    N4 += 0.5 * (abs(diag(real(da3))[j]) - diag(real(da3))[j])
end
L4 = log(1 + 2 * N4)


for j = 1:length(diag(real(da4)))
    N5 += 0.5 * (abs(diag(real(da4))[j]) - diag(real(da4))[j])
end
L5 = log(1 + 2 * N5)

for j = 1:length(diag(real(da5)))
    N6 += 0.5 * (abs(diag(real(da5))[j]) - diag(real(da5))[j])
end
L6 = log(1 + 2 * N6)


for j = 1:length(diag(real(da6)))
    N7 += 0.5 * (abs(diag(real(da6))[j]) - diag(real(da6))[j])
end
L7 = log(1 + 2 * N7)

L22=(L1*L2*L3*L4*L5*L6*L7)^(1/7)
@show L22;
    
    
end   

L22 = 0.10790263607343581


0.10790263607343581

In [18]:
using ITensors, ITensorMPS
ITensors.disable_warn_order()
let
A = 1
D = 7
N = 20

  sites = siteinds(n->isodd(n) ? "S=1/2" : "S=3/2",N)

  # Couplings between spin-half and
  # spin-one sites:
  J=1
  B=0.1

  ops = OpSum()
  for k=1:N-1
        ops += J,   "Sz",k, "Sz",k+1
        ops += J/2, "S+",k, "S-",k+1
        ops += J/2, "S-",k, "S+",k+1
    end
    
    #make it a ring
    ops += J,   "Sz",N, "Sz",1
    ops += J/2, "S+",N, "S-",1
    ops += J/2, "S-",N, "S+",1

    # spin-B interaction
    for k=1:N
        ops += -B, "Sz",k
    end
    
    H = MPO(ops,sites)

nsweeps = 20
maxdim = [5,50,100,150,200,250,300,400,500]
cutoff = [1E-11]

    
# Create the density matrix rho
a1 = 0
N1 = 0
a2 = 0
N2 = 0
a3 = 0
N3 = 0
a4 = 0
N4 = 0
a5 = 0
N5 = 0
a6 = 0
N6 = 0
a7 = 0
N7 = 0
orthogonalize!(ϕ,A)
ket1 = ϕ[A]
for i=A+1:D
    ket1 *= ϕ[i]
end
bra1 = dag(ket1)
rho = prime(ket1,"Site")*bra1
#@show inds(rho) 
rho_ABC = rho * delta(inds(rho)[3],inds(rho)[10])    
rho_ABC1 = rho_ABC*delta(inds(rho)[4],inds(rho)[11])   
rho_ABC3 = rho_ABC1*delta(inds(rho)[5],inds(rho)[12]) 


oldinds = inds(rho_ABC3)
newinds = [oldinds[5],oldinds[6],oldinds[3],oldinds[4],oldinds[1], oldinds[2], oldinds[7], oldinds[8]]
newinds1 = [oldinds[5],oldinds[2],oldinds[7],oldinds[4],oldinds[1], oldinds[6], oldinds[3], oldinds[8]]
newinds2 = [oldinds[5],oldinds[2],oldinds[3],oldinds[8],oldinds[1], oldinds[6], oldinds[7], oldinds[4]]
newinds3 = [oldinds[5],oldinds[2],oldinds[3],oldinds[4],oldinds[1], oldinds[6], oldinds[7], oldinds[8]]
newinds4 = [oldinds[1],oldinds[6],oldinds[3],oldinds[4],oldinds[5], oldinds[2], oldinds[7], oldinds[8]]
newinds5 = [oldinds[1],oldinds[2],oldinds[7],oldinds[4],oldinds[5], oldinds[6], oldinds[3], oldinds[8]]
newinds6 = [oldinds[1],oldinds[2],oldinds[3],oldinds[8],oldinds[5], oldinds[6], oldinds[7], oldinds[4]]


rho_pt = swapinds(rho_ABC3,oldinds,newinds)
rho_pt1 = swapinds(rho_ABC3,oldinds,newinds1)
rho_pt2 = swapinds(rho_ABC3,oldinds,newinds2)
rho_pt3 = swapinds(rho_ABC3,oldinds,newinds3)
rho_pt4 = swapinds(rho_ABC3,oldinds,newinds4)
rho_pt5 = swapinds(rho_ABC3,oldinds,newinds5)
rho_pt6 = swapinds(rho_ABC3,oldinds,newinds6)


da, ua = eigen(rho_pt)
da1, ua1 = eigen(rho_pt1)
da2, ua2 = eigen(rho_pt2)
da3, ua3 = eigen(rho_pt3)
da4, u4 = eigen(rho_pt4)
da5, ua5 = eigen(rho_pt5)
da6, ua6 = eigen(rho_pt6)

# Calculate the negativity
for j = 1:length(diag(real(da)))
    a1 += diag(real(da))[j]
end


for j = 1:length(diag(real(da)))
    N1 += 0.5 * (abs(diag(real(da))[j]) - diag(real(da))[j])
end
L1 = log(1 + 2 * N1)


for j = 1:length(diag(real(da1)))
    N2 += 0.5 * (abs(diag(real(da1))[j]) - diag(real(da1))[j])
end
L2 = log(1 + 2 * N2)

for j = 1:length(diag(real(da2)))
    N3 += 0.5 * (abs(diag(real(da2))[j]) - diag(real(da2))[j])
end
L3 = log(1 + 2 * N3)


for j = 1:length(diag(real(da3)))
    N4 += 0.5 * (abs(diag(real(da3))[j]) - diag(real(da3))[j])
end
L4 = log(1 + 2 * N4)


for j = 1:length(diag(real(da4)))
    N5 += 0.5 * (abs(diag(real(da4))[j]) - diag(real(da4))[j])
end
L5 = log(1 + 2 * N5)

for j = 1:length(diag(real(da5)))
    N6 += 0.5 * (abs(diag(real(da5))[j]) - diag(real(da5))[j])
end
L6 = log(1 + 2 * N6)


for j = 1:length(diag(real(da6)))
    N7 += 0.5 * (abs(diag(real(da6))[j]) - diag(real(da6))[j])
end
L7 = log(1 + 2 * N7)

L33=(L1*L2*L3*L4*L5*L6*L7)^(1/7)
@show L33;
    
    
end   

L33 = 0.0812356806514496


0.0812356806514496

In [19]:
using ITensors, ITensorMPS
ITensors.disable_warn_order()
let
A = 1
D = 8
N = 20

  sites = siteinds(n->isodd(n) ? "S=1/2" : "S=3/2",N)

  # Couplings between spin-half and
  # spin-one sites:
  J=1
  B=0.1

  ops = OpSum()
  for k=1:N-1
        ops += J,   "Sz",k, "Sz",k+1
        ops += J/2, "S+",k, "S-",k+1
        ops += J/2, "S-",k, "S+",k+1
    end
    
    #make it a ring
    ops += J,   "Sz",N, "Sz",1
    ops += J/2, "S+",N, "S-",1
    ops += J/2, "S-",N, "S+",1

    # spin-B interaction
    for k=1:N
        ops += -B, "Sz",k
    end
    
    H = MPO(ops,sites)


nsweeps = 20
maxdim = [5,50,100,150,200,250,300,400,500]
cutoff = [1E-11]

    
# Create the density matrix rho
a1 = 0
N1 = 0
a2 = 0
N2 = 0
a3 = 0
N3 = 0
a4 = 0
N4 = 0
a5 = 0
N5 = 0
a6 = 0
N6 = 0
a7 = 0
N7 = 0
orthogonalize!(ϕ,A)
ket1 = ϕ[A]
for i=A+1:D
    ket1 *= ϕ[i]
end
bra1 = dag(ket1)
rho = prime(ket1,"Site")*bra1
#@show inds(rho) 
rho_ABC = rho * delta(inds(rho)[3],inds(rho)[11])    
rho_ABC1 = rho_ABC*delta(inds(rho)[4],inds(rho)[12])   
rho_ABC2 = rho_ABC1*delta(inds(rho)[5],inds(rho)[13]) 
rho_ABC3 = rho_ABC2*delta(inds(rho)[6],inds(rho)[14])

oldinds = inds(rho_ABC3)
newinds = [oldinds[5],oldinds[6],oldinds[3],oldinds[4],oldinds[1], oldinds[2], oldinds[7], oldinds[8]]
newinds1 = [oldinds[5],oldinds[2],oldinds[7],oldinds[4],oldinds[1], oldinds[6], oldinds[3], oldinds[8]]
newinds2 = [oldinds[5],oldinds[2],oldinds[3],oldinds[8],oldinds[1], oldinds[6], oldinds[7], oldinds[4]]
newinds3 = [oldinds[5],oldinds[2],oldinds[3],oldinds[4],oldinds[1], oldinds[6], oldinds[7], oldinds[8]]
newinds4 = [oldinds[1],oldinds[6],oldinds[3],oldinds[4],oldinds[5], oldinds[2], oldinds[7], oldinds[8]]
newinds5 = [oldinds[1],oldinds[2],oldinds[7],oldinds[4],oldinds[5], oldinds[6], oldinds[3], oldinds[8]]
newinds6 = [oldinds[1],oldinds[2],oldinds[3],oldinds[8],oldinds[5], oldinds[6], oldinds[7], oldinds[4]]


rho_pt = swapinds(rho_ABC3,oldinds,newinds)
rho_pt1 = swapinds(rho_ABC3,oldinds,newinds1)
rho_pt2 = swapinds(rho_ABC3,oldinds,newinds2)
rho_pt3 = swapinds(rho_ABC3,oldinds,newinds3)
rho_pt4 = swapinds(rho_ABC3,oldinds,newinds4)
rho_pt5 = swapinds(rho_ABC3,oldinds,newinds5)
rho_pt6 = swapinds(rho_ABC3,oldinds,newinds6)


da, ua = eigen(rho_pt)
da1, ua1 = eigen(rho_pt1)
da2, ua2 = eigen(rho_pt2)
da3, ua3 = eigen(rho_pt3)
da4, u4 = eigen(rho_pt4)
da5, ua5 = eigen(rho_pt5)
da6, ua6 = eigen(rho_pt6)

# Calculate the negativity
for j = 1:length(diag(real(da)))
    a1 += diag(real(da))[j]
end


for j = 1:length(diag(real(da)))
    N1 += 0.5 * (abs(diag(real(da))[j]) - diag(real(da))[j])
end
L1 = log(1 + 2 * N1)


for j = 1:length(diag(real(da1)))
    N2 += 0.5 * (abs(diag(real(da1))[j]) - diag(real(da1))[j])
end
L2 = log(1 + 2 * N2)

for j = 1:length(diag(real(da2)))
    N3 += 0.5 * (abs(diag(real(da2))[j]) - diag(real(da2))[j])
end
L3 = log(1 + 2 * N3)


for j = 1:length(diag(real(da3)))
    N4 += 0.5 * (abs(diag(real(da3))[j]) - diag(real(da3))[j])
end
L4 = log(1 + 2 * N4)


for j = 1:length(diag(real(da4)))
    N5 += 0.5 * (abs(diag(real(da4))[j]) - diag(real(da4))[j])
end
L5 = log(1 + 2 * N5)

for j = 1:length(diag(real(da5)))
    N6 += 0.5 * (abs(diag(real(da5))[j]) - diag(real(da5))[j])
end
L6 = log(1 + 2 * N6)


for j = 1:length(diag(real(da6)))
    N7 += 0.5 * (abs(diag(real(da6))[j]) - diag(real(da6))[j])
end
L7 = log(1 + 2 * N7)

L44=(L1*L2*L3*L4*L5*L6*L7)^(1/7)
@show L44;
    
    
end   

L44 = 0.07612979398359769


0.07612979398359769

In [20]:
using ITensors, ITensorMPS
ITensors.disable_warn_order()
let
A = 1
D = 9
N = 20

  sites = siteinds(n->isodd(n) ? "S=1/2" : "S=3/2",N)

  # Couplings between spin-half and
  # spin-one sites:
  J=1
  B=0.1

  ops = OpSum()
  for k=1:N-1
        ops += J,   "Sz",k, "Sz",k+1
        ops += J/2, "S+",k, "S-",k+1
        ops += J/2, "S-",k, "S+",k+1
    end
    
    #make it a ring
    ops += J,   "Sz",N, "Sz",1
    ops += J/2, "S+",N, "S-",1
    ops += J/2, "S-",N, "S+",1

    # spin-B interaction
    for k=1:N
        ops += -B, "Sz",k
    end
    
    H = MPO(ops,sites)


nsweeps = 20
maxdim = [5,50,100,150,200,250,300,400,500]
cutoff = [1E-11]

    
# Create the density matrix rho
a1 = 0
N1 = 0
a2 = 0
N2 = 0
a3 = 0
N3 = 0
a4 = 0
N4 = 0
a5 = 0
N5 = 0
a6 = 0
N6 = 0
a7 = 0
N7 = 0
orthogonalize!(ϕ,A)
ket1 = ϕ[A]
for i=A+1:D
    ket1 *= ϕ[i]
end
bra1 = dag(ket1)
rho = prime(ket1,"Site")*bra1
#@show inds(rho) 
rho_ABC = rho * delta(inds(rho)[3],inds(rho)[12])    
rho_ABC1 = rho_ABC*delta(inds(rho)[4],inds(rho)[13])   
rho_ABC2 = rho_ABC1*delta(inds(rho)[5],inds(rho)[14]) 
rho_ABC21 = rho_ABC2*delta(inds(rho)[6],inds(rho)[15])
rho_ABC3 = rho_ABC21*delta(inds(rho)[7],inds(rho)[16])

oldinds = inds(rho_ABC3)
newinds = [oldinds[5],oldinds[6],oldinds[3],oldinds[4],oldinds[1], oldinds[2], oldinds[7], oldinds[8]]
newinds1 = [oldinds[5],oldinds[2],oldinds[7],oldinds[4],oldinds[1], oldinds[6], oldinds[3], oldinds[8]]
newinds2 = [oldinds[5],oldinds[2],oldinds[3],oldinds[8],oldinds[1], oldinds[6], oldinds[7], oldinds[4]]
newinds3 = [oldinds[5],oldinds[2],oldinds[3],oldinds[4],oldinds[1], oldinds[6], oldinds[7], oldinds[8]]
newinds4 = [oldinds[1],oldinds[6],oldinds[3],oldinds[4],oldinds[5], oldinds[2], oldinds[7], oldinds[8]]
newinds5 = [oldinds[1],oldinds[2],oldinds[7],oldinds[4],oldinds[5], oldinds[6], oldinds[3], oldinds[8]]
newinds6 = [oldinds[1],oldinds[2],oldinds[3],oldinds[8],oldinds[5], oldinds[6], oldinds[7], oldinds[4]]


rho_pt = swapinds(rho_ABC3,oldinds,newinds)
rho_pt1 = swapinds(rho_ABC3,oldinds,newinds1)
rho_pt2 = swapinds(rho_ABC3,oldinds,newinds2)
rho_pt3 = swapinds(rho_ABC3,oldinds,newinds3)
rho_pt4 = swapinds(rho_ABC3,oldinds,newinds4)
rho_pt5 = swapinds(rho_ABC3,oldinds,newinds5)
rho_pt6 = swapinds(rho_ABC3,oldinds,newinds6)


da, ua = eigen(rho_pt)
da1, ua1 = eigen(rho_pt1)
da2, ua2 = eigen(rho_pt2)
da3, ua3 = eigen(rho_pt3)
da4, u4 = eigen(rho_pt4)
da5, ua5 = eigen(rho_pt5)
da6, ua6 = eigen(rho_pt6)

# Calculate the negativity
for j = 1:length(diag(real(da)))
    a1 += diag(real(da))[j]
end


for j = 1:length(diag(real(da)))
    N1 += 0.5 * (abs(diag(real(da))[j]) - diag(real(da))[j])
end
L1 = log(1 + 2 * N1)


for j = 1:length(diag(real(da1)))
    N2 += 0.5 * (abs(diag(real(da1))[j]) - diag(real(da1))[j])
end
L2 = log(1 + 2 * N2)

for j = 1:length(diag(real(da2)))
    N3 += 0.5 * (abs(diag(real(da2))[j]) - diag(real(da2))[j])
end
L3 = log(1 + 2 * N3)


for j = 1:length(diag(real(da3)))
    N4 += 0.5 * (abs(diag(real(da3))[j]) - diag(real(da3))[j])
end
L4 = log(1 + 2 * N4)


for j = 1:length(diag(real(da4)))
    N5 += 0.5 * (abs(diag(real(da4))[j]) - diag(real(da4))[j])
end
L5 = log(1 + 2 * N5)

for j = 1:length(diag(real(da5)))
    N6 += 0.5 * (abs(diag(real(da5))[j]) - diag(real(da5))[j])
end
L6 = log(1 + 2 * N6)


for j = 1:length(diag(real(da6)))
    N7 += 0.5 * (abs(diag(real(da6))[j]) - diag(real(da6))[j])
end
L7 = log(1 + 2 * N7)

L55=(L1*L2*L3*L4*L5*L6*L7)^(1/7)
@show L55;
    
    
end   

L55 = 0.06911532008758672


0.06911532008758672

In [ ]:
using ITensors, ITensorMPS
ITensors.disable_warn_order()
let
A = 1
D = 10
N = 20

  sites = siteinds(n->isodd(n) ? "S=1/2" : "S=3/2",N)

  # Couplings between spin-half and
  # spin-one sites:
  J=1
  B=0.1

  ops = OpSum()
  for k=1:N-1
        ops += J,   "Sz",k, "Sz",k+1
        ops += J/2, "S+",k, "S-",k+1
        ops += J/2, "S-",k, "S+",k+1
    end
    
    #make it a ring
    ops += J,   "Sz",N, "Sz",1
    ops += J/2, "S+",N, "S-",1
    ops += J/2, "S-",N, "S+",1

    # spin-B interaction
    for k=1:N
        ops += -B, "Sz",k
    end
    
    H = MPO(ops,sites)


nsweeps = 20
maxdim = [5,50,100,150,200,250,300,400,500]
cutoff = [1E-11]

    
# Create the density matrix rho
a1 = 0
N1 = 0
a2 = 0
N2 = 0
a3 = 0
N3 = 0
a4 = 0
N4 = 0
a5 = 0
N5 = 0
a6 = 0
N6 = 0
a7 = 0
N7 = 0
orthogonalize!(ϕ,A)
ket1 = ϕ[A]
for i=A+1:D
    ket1 *= ϕ[i]
end
bra1 = dag(ket1)
rho = prime(ket1,"Site")*bra1
#@show inds(rho) 
rho_ABC = rho * delta(inds(rho)[3],inds(rho)[13])    
rho_ABC1 = rho_ABC*delta(inds(rho)[4],inds(rho)[14])   
rho_ABC2 = rho_ABC1*delta(inds(rho)[5],inds(rho)[15]) 
rho_ABC21 = rho_ABC2*delta(inds(rho)[6],inds(rho)[16])
rho_ABC22 = rho_ABC21*delta(inds(rho)[7],inds(rho)[17])
rho_ABC3 = rho_ABC22*delta(inds(rho)[8],inds(rho)[18])
    
oldinds = inds(rho_ABC3)
newinds = [oldinds[5],oldinds[6],oldinds[3],oldinds[4],oldinds[1], oldinds[2], oldinds[7], oldinds[8]]
newinds1 = [oldinds[5],oldinds[2],oldinds[7],oldinds[4],oldinds[1], oldinds[6], oldinds[3], oldinds[8]]
newinds2 = [oldinds[5],oldinds[2],oldinds[3],oldinds[8],oldinds[1], oldinds[6], oldinds[7], oldinds[4]]
newinds3 = [oldinds[5],oldinds[2],oldinds[3],oldinds[4],oldinds[1], oldinds[6], oldinds[7], oldinds[8]]
newinds4 = [oldinds[1],oldinds[6],oldinds[3],oldinds[4],oldinds[5], oldinds[2], oldinds[7], oldinds[8]]
newinds5 = [oldinds[1],oldinds[2],oldinds[7],oldinds[4],oldinds[5], oldinds[6], oldinds[3], oldinds[8]]
newinds6 = [oldinds[1],oldinds[2],oldinds[3],oldinds[8],oldinds[5], oldinds[6], oldinds[7], oldinds[4]]


rho_pt = swapinds(rho_ABC3,oldinds,newinds)
rho_pt1 = swapinds(rho_ABC3,oldinds,newinds1)
rho_pt2 = swapinds(rho_ABC3,oldinds,newinds2)
rho_pt3 = swapinds(rho_ABC3,oldinds,newinds3)
rho_pt4 = swapinds(rho_ABC3,oldinds,newinds4)
rho_pt5 = swapinds(rho_ABC3,oldinds,newinds5)
rho_pt6 = swapinds(rho_ABC3,oldinds,newinds6)


da, ua = eigen(rho_pt)
da1, ua1 = eigen(rho_pt1)
da2, ua2 = eigen(rho_pt2)
da3, ua3 = eigen(rho_pt3)
da4, u4 = eigen(rho_pt4)
da5, ua5 = eigen(rho_pt5)
da6, ua6 = eigen(rho_pt6)

# Calculate the negativity
for j = 1:length(diag(real(da)))
    a1 += diag(real(da))[j]
end


for j = 1:length(diag(real(da)))
    N1 += 0.5 * (abs(diag(real(da))[j]) - diag(real(da))[j])
end
L1 = log(1 + 2 * N1)


for j = 1:length(diag(real(da1)))
    N2 += 0.5 * (abs(diag(real(da1))[j]) - diag(real(da1))[j])
end
L2 = log(1 + 2 * N2)

for j = 1:length(diag(real(da2)))
    N3 += 0.5 * (abs(diag(real(da2))[j]) - diag(real(da2))[j])
end
L3 = log(1 + 2 * N3)


for j = 1:length(diag(real(da3)))
    N4 += 0.5 * (abs(diag(real(da3))[j]) - diag(real(da3))[j])
end
L4 = log(1 + 2 * N4)


for j = 1:length(diag(real(da4)))
    N5 += 0.5 * (abs(diag(real(da4))[j]) - diag(real(da4))[j])
end
L5 = log(1 + 2 * N5)

for j = 1:length(diag(real(da5)))
    N6 += 0.5 * (abs(diag(real(da5))[j]) - diag(real(da5))[j])
end
L6 = log(1 + 2 * N6)


for j = 1:length(diag(real(da6)))
    N7 += 0.5 * (abs(diag(real(da6))[j]) - diag(real(da6))[j])
end
L7 = log(1 + 2 * N7)

L66=(L1*L2*L3*L4*L5*L6*L7)^(1/7)
@show L66;
    
    
end   

In [ ]:
using ITensors, ITensorMPS
ITensors.disable_warn_order()
let
A = 1
D = 11
N = 20

  sites = siteinds(n->isodd(n) ? "S=1/2" : "S=3/2",N)

  # Couplings between spin-half and
  # spin-one sites:
  J=1
  B=0.1

  ops = OpSum()
  for k=1:N-1
        ops += J,   "Sz",k, "Sz",k+1
        ops += J/2, "S+",k, "S-",k+1
        ops += J/2, "S-",k, "S+",k+1
    end
    
    #make it a ring
    ops += J,   "Sz",N, "Sz",1
    ops += J/2, "S+",N, "S-",1
    ops += J/2, "S-",N, "S+",1

    # spin-B interaction
    for k=1:N
        ops += -B, "Sz",k
    end
    
    H = MPO(ops,sites)


nsweeps = 20
maxdim = [5,50,100,150,200,250,300,400,500]
cutoff = [1E-11]

    
# Create the density matrix rho
a1 = 0
N1 = 0
a2 = 0
N2 = 0
a3 = 0
N3 = 0
a4 = 0
N4 = 0
a5 = 0
N5 = 0
a6 = 0
N6 = 0
a7 = 0
N7 = 0
orthogonalize!(ϕ,A)
ket1 = ϕ[A]
for i=A+1:D
    ket1 *= ϕ[i]
end
bra1 = dag(ket1)
rho = prime(ket1,"Site")*bra1
#@show inds(rho) 
rho_ABC = rho * delta(inds(rho)[3],inds(rho)[14])    
rho_ABC1 = rho_ABC*delta(inds(rho)[4],inds(rho)[15])   
rho_ABC2 = rho_ABC1*delta(inds(rho)[5],inds(rho)[16]) 
rho_ABC21 = rho_ABC2*delta(inds(rho)[6],inds(rho)[17])
rho_ABC22 = rho_ABC21*delta(inds(rho)[7],inds(rho)[18])
rho_ABC23 = rho_ABC22*delta(inds(rho)[8],inds(rho)[19])
rho_ABC3 = rho_ABC23*delta(inds(rho)[9],inds(rho)[20])
    
oldinds = inds(rho_ABC3)
newinds = [oldinds[5],oldinds[6],oldinds[3],oldinds[4],oldinds[1], oldinds[2], oldinds[7], oldinds[8]]
newinds1 = [oldinds[5],oldinds[2],oldinds[7],oldinds[4],oldinds[1], oldinds[6], oldinds[3], oldinds[8]]
newinds2 = [oldinds[5],oldinds[2],oldinds[3],oldinds[8],oldinds[1], oldinds[6], oldinds[7], oldinds[4]]
newinds3 = [oldinds[5],oldinds[2],oldinds[3],oldinds[4],oldinds[1], oldinds[6], oldinds[7], oldinds[8]]
newinds4 = [oldinds[1],oldinds[6],oldinds[3],oldinds[4],oldinds[5], oldinds[2], oldinds[7], oldinds[8]]
newinds5 = [oldinds[1],oldinds[2],oldinds[7],oldinds[4],oldinds[5], oldinds[6], oldinds[3], oldinds[8]]
newinds6 = [oldinds[1],oldinds[2],oldinds[3],oldinds[8],oldinds[5], oldinds[6], oldinds[7], oldinds[4]]


rho_pt = swapinds(rho_ABC3,oldinds,newinds)
rho_pt1 = swapinds(rho_ABC3,oldinds,newinds1)
rho_pt2 = swapinds(rho_ABC3,oldinds,newinds2)
rho_pt3 = swapinds(rho_ABC3,oldinds,newinds3)
rho_pt4 = swapinds(rho_ABC3,oldinds,newinds4)
rho_pt5 = swapinds(rho_ABC3,oldinds,newinds5)
rho_pt6 = swapinds(rho_ABC3,oldinds,newinds6)


da, ua = eigen(rho_pt)
da1, ua1 = eigen(rho_pt1)
da2, ua2 = eigen(rho_pt2)
da3, ua3 = eigen(rho_pt3)
da4, u4 = eigen(rho_pt4)
da5, ua5 = eigen(rho_pt5)
da6, ua6 = eigen(rho_pt6)

# Calculate the negativity
for j = 1:length(diag(real(da)))
    a1 += diag(real(da))[j]
end


for j = 1:length(diag(real(da)))
    N1 += 0.5 * (abs(diag(real(da))[j]) - diag(real(da))[j])
end
L1 = log(1 + 2 * N1)


for j = 1:length(diag(real(da1)))
    N2 += 0.5 * (abs(diag(real(da1))[j]) - diag(real(da1))[j])
end
L2 = log(1 + 2 * N2)

for j = 1:length(diag(real(da2)))
    N3 += 0.5 * (abs(diag(real(da2))[j]) - diag(real(da2))[j])
end
L3 = log(1 + 2 * N3)


for j = 1:length(diag(real(da3)))
    N4 += 0.5 * (abs(diag(real(da3))[j]) - diag(real(da3))[j])
end
L4 = log(1 + 2 * N4)


for j = 1:length(diag(real(da4)))
    N5 += 0.5 * (abs(diag(real(da4))[j]) - diag(real(da4))[j])
end
L5 = log(1 + 2 * N5)

for j = 1:length(diag(real(da5)))
    N6 += 0.5 * (abs(diag(real(da5))[j]) - diag(real(da5))[j])
end
L6 = log(1 + 2 * N6)


for j = 1:length(diag(real(da6)))
    N7 += 0.5 * (abs(diag(real(da6))[j]) - diag(real(da6))[j])
end
L7 = log(1 + 2 * N7)

L77=(L1*L2*L3*L4*L5*L6*L7)^(1/7)
@show L77;
    
    
end   

In [ ]:
L00=0.4978856855988925; L11 = 0.23286051319570816; L22=0.10790263607343581; 
L33=0.0812356806514496; L44=0.07612979398359769; L55=0.06911532008758672 ; L66= ; L77=

In [ ]:
using Plots
using LaTeXStrings

k=[0,1,2,3,4,5,6,7]
La=[L00,L11,L22,L33,L44,L55,L66,L77]

plot(k,La)